**시간표 관리 프로그램 코드를 클래스를 활용해 객체지향 방식으로 변경하기/0610/20220785/이상권**

In [1]:
import pickle
import os
weekdays = ['월', '화', '수', '목', '금']
hours = [str(x) for x in range(1, 9)]
TimeTable = {key: {key: ' ' for key, value in dict.fromkeys(weekdays).items()} for key in dict.fromkeys(hours).keys()}

class Timetable():
    def __init__(self, day, period, relay, alphabet):
        self.day = day
        self.period = period
        self.relay = relay
        self.alphabet = alphabet

    def Weekend(self):
        print('>>> [ERROR] 토요일과 일요일은 수업이 없습니다!')
        print('>>> 다시 입력하세요.')

    def Period_Under_Exceed(self):
        print('>>> [ERROR] 등록하려는 수업이 등록 가능 시간을 벗어납니다!')
        print('>>> 다시 입력하세요.')

    def Overlap(self):
        print('>>> [ERROR] 해당 시간에 이미 수업이 있습니다!')
        print('>>> 다시 입력하세요.')

    def print_TimeTable(self, **classdict):
        print('\n')
        print('[ TimeTable ]')
        print('-------------')
        print('| |M|T|W|T|F|')
        print('-------------')
        for hour in classdict.keys():
            print('|', end='')
            print(hour, end='|')
            for day in classdict[hour].keys():
                print(classdict[hour][day], end='|')
            print()
        print('-------------\n')

    def registTimeTable(self, relay, day, alphabet, period):
        TimeTable[self.period][self.day] = self.alphabet
        if self.relay >= 2:
            while self.relay != 1:  # relay가 1이 될 때 까지 연강 시간 만큼 다음 행 동일 열에 알파벳 표시
                TimeTable[str(int(self.period) + (self.relay - 1))][self.day] = self.alphabet
                self.relay -= 1

    def saveTimeTabletxt(self, **classdict):
        f = open('timetable.txt', 'w')
        print('\n', file=f)
        print('[ TimeTable ]', file=f)
        print('-------------', file=f)
        print('| |M|T|W|T|F|', file=f)
        print('-------------', file=f)
        for hour in classdict.keys():
            print('|', end='', file=f)
            print(hour, end='|', file=f)
            for day in classdict[hour].keys():
                print(classdict[hour][day], end='|', file=f)
            print(file=f)
        print('-------------\n', file=f)
        f.close()

    def saveTimeTableDic(self, **classdict):
        with open('timetable.pkl', 'wb') as f:
            pickle.dump(classdict, f)

    def loadTimeTableTxt(self):
        with open('timetable.txt', 'r') as file:
            for text in file:
                print(text.strip('\n'))

    def loadTimbeTableDic(self):
        with open('timetable.pkl', 'rb') as f:
            mydict = pickle.load(f)
            tb.print_TimeTable(**mydict)
            
tb1 = Timetable(0, 0, 0, ' ')
tb1.print_TimeTable(**TimeTable)

# 시간표 등록
while True:
    # 요일 및 교시 입력
    day, period = input('>>> 등록할 수업의 요일과 교시를 입력하세요:').split()
    # 연강 시간 입력
    relay = int(input('>>> 등록할 수업의 연강 시간을 입력하세요:'))
    # 알파벳 입력
    alphabet = input('>>> 등록할 수업의 알파벳을 입력하세요:')
    # 시간표 생성
    tb = Timetable(day, period, relay, alphabet)

    # 토요일 혹은 일요일은 예외
    if day == '토' or day == '일':
        tb.Weekend()
        continue

    # 0교시나 8교시 초과 시 예외
    elif int(period) <= 0 or int(period) > 8:
        tb.Period_Under_Exceed()
        continue

    else:
        # 시간표 중복 시 예외
        if TimeTable[period][day] != ' ':
            tb.Overlap()
            continue

        else:  # 시간표 등록
            tb.registTimeTable(relay, day, alphabet, period)

    tb.print_TimeTable(**TimeTable)

    # 시간표 등록을 계속하거나 종료
    ex_it = int(input('>>> 시간표 등록을 계속하시겠습니까?    1. 예    2. 아니요    '))
    if ex_it == 2:
        print('>>> 시간표 등록을 종료합니다.')
        break

save = int(input('>>> 등록된 시간표를 저장하시겠습니까?    1. 예    2.아니요    '))
if save == 1:
    save_sel = int(
        input('>>> 저장할 방식을 선택하세요:    1. 텍스트 저장    2. 딕셔너리 저장    3. 둘 다 저장 선택하기    '))
    if save_sel == 1:
        tb.saveTimeTabletxt(**TimeTable)
        print('>>> 텍스트로 저장되었습니다.')

    elif save_sel == 2:
        tb.saveTimeTableDic(**TimeTable)
        print('>>> 딕셔너리로 저장되었습니다.')

    elif save_sel == 3:
        tb.saveTimeTabletxt(**TimeTable)
        tb.saveTimeTableDic(**TimeTable)
        print('>>> 텍스트와 딕셔너리 둘 다 저장되었습니다.')

load = int(input('>>> 저장된 시간표를 불러오시겠습니까?    1. 예    2.아니요    '))
if load == 1:
    load_sel = int(
        input('>>> 불러올 파일을 선택하세요:    1. 텍스트 불러오기    2. 딕셔너리 불러오기    '))
    if load_sel == 1:
        if(os.path.isfile('timetable.txt')):
            tb.loadTimeTableTxt()

        else:
            print('>>> [ERROR] 불러올 시간표가 없습니다!')

    elif load_sel == 2:
        if (os.path.isfile('timetable.pkl')):
            tb.loadTimbeTableDic()

        else:
            print('>>> [ERROR] 불러올 시간표가 없습니다!')



[ TimeTable ]
-------------
| |M|T|W|T|F|
-------------
|1| | | | | |
|2| | | | | |
|3| | | | | |
|4| | | | | |
|5| | | | | |
|6| | | | | |
|7| | | | | |
|8| | | | | |
-------------

>>> 등록할 수업의 요일과 교시를 입력하세요:목 1
>>> 등록할 수업의 연강 시간을 입력하세요:3
>>> 등록할 수업의 알파벳을 입력하세요:W


[ TimeTable ]
-------------
| |M|T|W|T|F|
-------------
|1| | | |W| |
|2| | | |W| |
|3| | | |W| |
|4| | | | | |
|5| | | | | |
|6| | | | | |
|7| | | | | |
|8| | | | | |
-------------

>>> 시간표 등록을 계속하시겠습니까?    1. 예    2. 아니요    1
>>> 등록할 수업의 요일과 교시를 입력하세요:금 2
>>> 등록할 수업의 연강 시간을 입력하세요:2
>>> 등록할 수업의 알파벳을 입력하세요:R


[ TimeTable ]
-------------
| |M|T|W|T|F|
-------------
|1| | | |W| |
|2| | | |W|R|
|3| | | |W|R|
|4| | | | | |
|5| | | | | |
|6| | | | | |
|7| | | | | |
|8| | | | | |
-------------

>>> 시간표 등록을 계속하시겠습니까?    1. 예    2. 아니요    1
>>> 등록할 수업의 요일과 교시를 입력하세요:화 4
>>> 등록할 수업의 연강 시간을 입력하세요:2
>>> 등록할 수업의 알파벳을 입력하세요:H


[ TimeTable ]
-------------
| |M|T|W|T|F|
-------------
|1| | | |W| |
|2| | | |W|R|
|3| | | |W|R|
|4| |H| |

**코딩 후기: 메소드 생성이 꽤 헷갈렸다.**